In [2]:
import pandas as pd
import os

In [3]:
#change this variable from 2001 to 2012 one at a time
year = 2001

In [52]:
dirs = os.listdir('{}'.format(year))
d = dict()
for i,files in zip(map(str, range(1, len(dirs)+1)), dirs):
    if files.startswith('all_contracts'):
        d['df_'+i] = pd.read_csv("{}/".format(year) + files)
df = pd.concat([d[i] for i in d.keys()], ignore_index=True).reset_index()

/Users/Christine/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,37,47,49,50,57,66,75,76,89,109,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Christine/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (37,39,47,49,50,57,66,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Christine/anaconda3/envs/py2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (3,37,39,43,47,49,50,57,66,75,76,89,109,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
df['recipient_city_state'] = df['recipient_city_name'] + ', ' + df['recipient_state_code']
df['performance_city_state'] = df['primary_place_of_performance_city_name'] + ', '  \
                               + df['primary_place_of_performance_state_code']
df_city = df[['awarding_agency_name', 'federal_action_obligation', 'recipient_city_name', 
              'recipient_state_code', 'recipient_city_state', 'primary_place_of_performance_city_name', 
              'primary_place_of_performance_state_code', 'performance_city_state']]

In [55]:
df_recipient = df_city.groupby('recipient_city_state') \
.agg({'federal_action_obligation':'sum', 'awarding_agency_name':'count'}) \
.rename(columns={'awarding_agency_name':'recipient_count', 'federal_action_obligation':'recipient_amount'}) \
.reset_index()

df_performance = df_city.groupby('performance_city_state') \
.agg({'federal_action_obligation':'sum', 'awarding_agency_name':'count'}) \
.rename(columns={'awarding_agency_name':'performance_count', 'federal_action_obligation':'performance_amount'}) \
.reset_index()

df_recipient = df_recipient.merge(df_city[['recipient_city_name','recipient_state_code',
                                           'recipient_city_state']],
                                  on='recipient_city_state').drop_duplicates().reset_index(drop=True)

df_performance = df_performance.merge(df_city[['performance_city_state', 
                                               'primary_place_of_performance_city_name', 
                                               'primary_place_of_performance_state_code']],
                                      on='performance_city_state').drop_duplicates().reset_index(drop=True)

In [60]:
df_performance.to_csv('FF{}_performance.csv'.format(year))
df_recipient.to_csv('FF{}_recipient.csv'.format(year))